In [333]:
import glob
import os
import numpy as np
import cv2
import imutils
import math
from scipy import misc
import copy

In [334]:
img_dir = "dataset"

data_path = os.path.join(img_dir,'*.jpg')
files = glob.glob(data_path)
print(files)

['dataset\\087_l_940_05.jpg']


In [335]:
class_data = []
dir_result = "result_ROI"
dir_region = "region_ROI"
for f1 in files:
    original_img = cv2.imread(f1)
    head, tail = os.path.split(f1)
    name = tail.split('_')
    class_data.append(name[0])
    folderName = dir_result + '\\\\' + tail
    
    #rotate image
    img_rotate = imutils.rotate_bound(original_img, 90)
    
    #graylevel image
    img_gray = cv2.cvtColor(img_rotate, cv2.COLOR_BGR2GRAY)
    
    #otsu's thresholding
    blur = cv2.GaussianBlur(img_gray,(5,5),0)
    ret3,img_thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    #morphological opening
    sz = 20
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*sz-1, 2*sz-1))
    opening = cv2.morphologyEx(img_thresh, cv2.MORPH_OPEN, kernel)
    
    cv2.imwrite(os.path.join(dir_result, tail), opening)
    

In [336]:
img = np.array(opening)
column = np.argwhere(np.any(img==255, axis = 0))[0]
c = column[0]
row_columnX = img[:,c:c+1]
print(row_columnX.shape)
row = np.argwhere(np.any(row_columnX==255, axis = 1))[0]
r = row[0]
print(c)
print(r)
ctrImg = np.zeros(img.shape)
print(img.shape)
height, width = img.shape
ctrImg[r,c] = 255
coor = [r,c]
startCoor = [r,c]
print(ctrImg[r,c])
print(coor)
try_img = np.zeros(img.shape)
for i in range(height - r):
    for j in range(width - c):
        try_img[i+r][j+c] = 255
# cv2.imwrite(os.path.join(dir_result, "try.jpg"), try_img)

(768, 1)
51
591
(768, 576)
255.0
[591, 51]


In [337]:
maksCoorR, maksCoorC = img.shape
coorHalfWrist = [int(maksCoorR), int(maksCoorC/2)]
print(coorHalfWrist)
try1_img = np.zeros(img.shape)
for i in range(coorHalfWrist[0]):
    for j in range(coorHalfWrist[1]):
        try1_img[i][j] = 255
# cv2.imwrite(os.path.join(dir_result, "try1.jpg"), try1_img)

[768, 288]


In [338]:
newCoor = [0,0]
coorDis = []
direction = []
dire = 7
indikator = 1
count = 0
counts = 0
while indikator == 1:
    counts += 1
    if dire % 2 == 0:
        startDir = (dire+7) % 8
    else:
        startDir = (dire+6) % 8
    currentDir = startDir
    for i in range(8):
        if currentDir == 8:
            currentDir = 0
        if currentDir == 0:
            posisi = [0, 1]
        elif currentDir == 1:
            posisi = [-1, 1]
        elif currentDir == 2:
            posisi = [-1, 0]
        elif currentDir == 3:
            posisi = [-1, -1]
        elif currentDir == 4:
            posisi = [0, -1]
        elif currentDir == 5:
            posisi = [1, -1]
        elif currentDir == 6:
            posisi = [1, 0]
        elif currentDir == 7:
            posisi = [1, 1]
        dire = currentDir
#         print("posisi")
#         print(posisi)
#         print("coor")
#         print(coor)
        newCoor[0] = coor[0] + posisi[0]
        newCoor[1] = coor[1] + posisi[1]
#         print("newCoor")
#         print(newCoor)
        if newCoor[0] >= 0 and newCoor[1] >= 0 and newCoor[0] < maksCoorR-1 and newCoor[1] < maksCoorC-1:
            if img[newCoor[0], newCoor[1]] == 255:
                ctrImg[newCoor[0], newCoor[1]] = 255
                coor[0] = newCoor[0]
                coor[1] = newCoor[1]
#                 print(newCoor)
                a = math.pow((newCoor[0] - coorHalfWrist[0] + 1),2)
                b = math.pow((newCoor[1] - coorHalfWrist[1] + 1),2)
                c = a + b
#                 print(c)
                distances = math.sqrt(c)
                coorDis.append([])
                direction.append([])
                coorDis[count].append(newCoor[0])
                coorDis[count].append(newCoor[1])
                coorDis[count].append(distances)
                direction[count].append(dire)
                count += 1
                break
            else:
                currentDir += 1
        else:
            currentDir += 1
    if startCoor == coor:
        indikator = 0

In [339]:
cv2.imwrite(os.path.join(dir_result, tail), ctrImg)

True

In [340]:
type(coorDis)

list

In [341]:
# Distance Distribution Diagram
sumbuX = []
sumbuX = np.arange(len(coorDis))
sumbuX = np.array(sumbuX)
print(type(coorDis[0][0]))
print(type(np.amin(coorDis, axis =0)[0]))
minDistance = np.amin(coorDis, axis =0)[2]
indexDistance = np.argwhere(coorDis == minDistance)
for i in indexDistance:
    if i[1] == 2:
        r = i[0]
        print(coorDis[r])
        break
c = 0
height, width = img.shape
print(indexDistance)
print(minDistance)
a = coorDis[indexDistance[0][0]][0]
b = coorDis[indexDistance[0][0]][1]
print(r)

<class 'numpy.int64'>
<class 'numpy.float64'>
[729, 266, 43.41658669218482]
[[236   2]]
43.4165866922
236


In [342]:
newCoorDis = []
for i in range(len(coorDis) - r):
    newCoorDis.append(coorDis[i+r])
for i in range(r):
    newCoorDis.append(coorDis[i])
newCoorDis = np.array(newCoorDis)
height, width = ctrImg.shape
image = ctrImg
new = newCoorDis[0]
# print(newCoorDis)
# for i in range(150):
#     image[int(new[0])][int(new[1]) + i] = 255
# cv2.imwrite(os.path.join(dir_result, tail), image)
print(len(newCoorDis))

2562


In [343]:
whiteDis = float(550)
r = []
c = []
for i in range(len(newCoorDis)):
    if newCoorDis[i][0] > whiteDis:
        newCoorDis[i][2] = 0
        ctrImg[int(newCoorDis[i][0]), int(newCoorDis[i][1])] = 0
distancess = []
for i in range(len(newCoorDis)):
    distancess.append(newCoorDis[i][2])
# distancess = np.array(distancess)
print(distancess)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [344]:
cv2.imwrite(os.path.join(dir_result, tail), ctrImg)

True

In [345]:
def getPeak(Data, i, s):
    if (i-s) < 1:
        top = 0
    else:
        top = i - s - 1
    y = Data[1]
    
    if (i+s) > len(y):
        bottom = len(y) - 1
    else:
        bottom = i + s - 1
    p = []
    if y[i] == max(y[top:bottom]) or y[i] == min(y[top:bottom]):
        p = [Data[0][i], Data[1][i]]
#         if Data[1][i] > 0.0:
#             temp = 1
    else:
        p.append(float('nan'))
        p.append(float('nan'))
    return p

In [346]:
def fpeak(sumbuX, distancess, s):
    rx = len(sumbuX)
    ry = len(distancess)
    if s == 0:
        s = 1
    Data = np.vstack((sumbuX, distancess))
#     print(Data)
    output = []
    for i in range(rx):
        isP= getPeak(Data, i, s)
#         print(isP)
        if np.isnan(isP).sum() == 0:
            output.append(isP)
#             if temp == 1:
#                 print("hola")
    return output

In [347]:
s = 50
# print(len(distancess))
output = fpeak(sumbuX, distancess, s)
print(output)
for i in range(len(output)):
    if i == len(output) - 1:
        output[i].append(0)
        break
    if output[i][1] == output[i+1][1]:
        output[i].append(1)
    else:
        output[i].append(0)
#         print("hahaha")
# print(output)

[[0.0, 0.0], [1.0, 0.0], [2.0, 0.0], [3.0, 0.0], [4.0, 0.0], [5.0, 0.0], [6.0, 0.0], [7.0, 0.0], [8.0, 0.0], [9.0, 0.0], [10.0, 0.0], [11.0, 0.0], [12.0, 0.0], [13.0, 0.0], [14.0, 0.0], [15.0, 0.0], [16.0, 0.0], [17.0, 0.0], [18.0, 0.0], [19.0, 0.0], [20.0, 0.0], [21.0, 0.0], [22.0, 0.0], [23.0, 0.0], [24.0, 0.0], [25.0, 0.0], [26.0, 0.0], [27.0, 0.0], [28.0, 0.0], [29.0, 0.0], [30.0, 0.0], [31.0, 0.0], [32.0, 0.0], [33.0, 0.0], [34.0, 0.0], [35.0, 0.0], [36.0, 0.0], [37.0, 0.0], [38.0, 0.0], [39.0, 0.0], [40.0, 0.0], [41.0, 0.0], [42.0, 0.0], [43.0, 0.0], [44.0, 0.0], [45.0, 0.0], [46.0, 0.0], [47.0, 0.0], [48.0, 0.0], [49.0, 0.0], [50.0, 0.0], [51.0, 0.0], [52.0, 0.0], [53.0, 0.0], [54.0, 0.0], [55.0, 0.0], [56.0, 0.0], [57.0, 0.0], [58.0, 0.0], [59.0, 0.0], [60.0, 0.0], [61.0, 0.0], [62.0, 0.0], [63.0, 0.0], [64.0, 0.0], [65.0, 0.0], [66.0, 0.0], [67.0, 0.0], [68.0, 0.0], [69.0, 0.0], [70.0, 0.0], [71.0, 0.0], [72.0, 0.0], [73.0, 0.0], [74.0, 0.0], [75.0, 0.0], [76.0, 0.0], [77.0, 0

In [348]:
improveOut = []
for i in output:
    if i[2] == float(0):
        improveOut.append(i)
# sorted(improveOut, reverse=True)
output = improveOut
tails = tail.split('.')
cv2.imwrite(os.path.join(dir_result, tail), ctrImg)
for i in output:
    new = newCoorDis[int(i[0])]
    images = copy.deepcopy(ctrImg)
    for j in range(30):
#         print(int(new[0])+j)
#         print(int(new[1]))
        image[int(new[0])+j][int(new[1])] = 255
    name = tails[0] + '_' + str(i[0]) + '.jpg'
    cv2.imwrite(os.path.join(dir_result, name), image)

In [182]:
# print(outpuT)
pixelP1 = output[0][0]
pixelP2 = output[6][0]
print(pixelP1)
coorP1 = [newCoorDis[int(pixelP1)][0], newCoorDis[int(pixelP1)][1]]
coorP2 = [newCoorDis[int(pixelP2)][0], newCoorDis[int(pixelP2)][1]]
print(coorP1)
print(pixelP2)
print(coorP2)

277.0
[551.0, 466.0]
785.0
[438.0, 281.0]


In [17]:
# kemiringan p1 dam p2
temp = (coorP2[1] - coorP1[1]) / (coorP2[0] - coorP1[0])
print(temp)
deg = math.atan(temp) / math.pi * 180
print(deg)

0.970760233918
44.149976618908916


In [18]:
newImg = misc.imrotate(img_gray,90 - deg,interp='bilinear')
# import skimage.transform
# newImg = skimage.transform.rotate(img_gray, 90-deg)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `imrotate` is deprecated!
`imrotate` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.rotate`` instead.
  """Entry point for launching an IPython kernel.


In [19]:
cv2.imwrite(os.path.join(dir_result, tail), newImg)

True

In [20]:
temp = np.zeros_like(img)
height, width = img.shape
temp[int(coorP1[0])][int(coorP1[1])] = 255
temp[int(coorP2[0])][int(coorP2[1])] = 255
# for i in range(width - int(coorP1[1])):
#     img[int(coorP1[0])][int(coorP1[1])-i] = 0
# for i in range(width - int(coorP2[1])):
#     img[int(coorP2[0])][int(coorP2[1])+i] = 0

In [24]:
tempRotate = misc.imrotate(temp,90 - deg,interp='bilinear')
print(tempRotate)

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `imrotate` is deprecated!
`imrotate` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.rotate`` instead.
  """Entry point for launching an IPython kernel.


In [25]:
cv2.imwrite(os.path.join(dir_result, tail), tempRotate)

True

In [26]:
tempIndex = np.argwhere(tempRotate>0)
print(tempIndex)

[[360 305]
 [360 543]
 [361 304]
 [361 305]
 [361 543]
 [361 544]]


In [27]:
newCoorP1 = [tempIndex[0][0], tempIndex[0][1]]
newCoorP2 = [tempIndex[len(tempIndex)-1][0], tempIndex[len(tempIndex)-1][1]]
print(newCoorP1)
print(newCoorP2)

[360, 305]
[361, 544]


In [28]:
a = math.pow((newCoorP1[0]-newCoorP2[0]),2)
b = math.pow((newCoorP1[1]-newCoorP2[1]),2)
distance = math.sqrt(a+b)
distance = int(distance)
            
print(distance)
# ROI output
print(int(newCoorP1[0]+distance))
print(int(newCoorP1[1]+distance))
fiturI = newImg[int(newCoorP1[0]):int(newCoorP1[0]+distance), int(newCoorP1[1]):int(newCoorP1[1]+distance)]
cv2.imwrite(os.path.join(dir_result, tail), fiturI)


239
599
544


True

In [29]:
for a in range(distance):
    for b in range(distance):
        if (a+newCoorP1[0]) < height and (b+newCoorP1[1]) < width:
            newImg[a+newCoorP1[0]][b+newCoorP1[1]] = 255
cv2.imwrite(os.path.join(dir_region, tail), newImg)

True